In [ ]:
from sklearn.datasets import fetch_mldata
from sklearn import model_selection

from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
#from mnist import MNIST
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve
from sklearn.svm import LinearSVC
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import StandardScaler, Normalizer

In [ ]:
mnist = fetch_mldata('MNIST original')
X, y = mnist["data"], mnist["target"]

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
X_tr, X_val, y_tr, y_val = train_test_split(X_train,y_train,test_size=0.2)

In [ ]:
shuffle_index = np.random.permutation(48000)
X_tr, y_tr = X_tr[shuffle_index], y_tr[shuffle_index]

In [ ]:
some_digit = X_tr[1784]
some_digit_image = some_digit.reshape(28, 28)
plt.imshow(some_digit_image, cmap = matplotlib.cm.binary,interpolation="nearest")
plt.show()


In [ ]:
clf_rf = RandomForestClassifier(n_estimators=200, n_jobs=10,)
clf_rf.fit(X_tr, y_tr)

In [ ]:
clf_rf.predict([some_digit])

In [ ]:
cross_val_score(clf_rf, X_val, y_val, cv=3, scoring="accuracy")

In [ ]:
scaler = Normalizer()
X_tr_scaled = scaler.fit_transform(X_tr.astype(np.float64))

In [ ]:
cr_val = cross_val_score(clf_rf, X_tr_scaled, y_tr, cv=3, scoring="accuracy")
y_tr_pred = cross_val_predict(clf_rf, X_tr_scaled, y_tr, cv=3)

In [ ]:
confusion_matrix = confusion_matrix(y_tr, y_tr_pred)
print(confusion_matrix)

In [ ]:
cross_val_score(clf_rf, X_test, y_test, cv=3, scoring="accuracy")

In [ ]:
def plot_confusion_matrix(matrix):
    """If you prefer color and a colorbar"""
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    cax = ax.matshow(matrix)
    fig.colorbar(cax)

plt.matshow(confusion_matrix, cmap=plt.cm.gray)
plt.show()

row_sums = confusion_matrix.sum(axis=1, keepdims=True)
norm_conf_mx = confusion_matrix / row_sums
np.fill_diagonal(norm_conf_mx, 0)
plt.matshow(norm_conf_mx, cmap=plt.cm.gray)
plt.show()

In [ ]:
y_rf_pred = clf_rf.predict(X_test)
accuracy_score(y_test, y_rf_pred)

In [ ]:
print (precision_score(y_tr, y_tr_pred, average = None))
print (precision_score(y_tr, y_tr_pred, average = 'weighted'))


In [ ]:
print(recall_score(y_tr, y_tr_pred, average = None))
print(recall_score(y_tr, y_tr_pred, average = 'weighted'))

In [ ]:
print(f1_score(y_tr, y_tr_pred, average = None))
print(f1_score(y_tr, y_tr_pred, average = 'weighted'))

In [ ]:
y_train_all = (y_tr == [0 or 1 or 2 or 3 or 4 or 5 or 6 or 7 or 8 or 9])
y_train_all

In [ ]:
y_scores = cross_val_predict(clf_rf, X_tr, y_train_all, cv=3)

In [ ]:
if y_scores.ndim == 2:
    y_scores = y_scores[:, 1]

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(y_train_all, y_scores)
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="upper left", fontsize=16)
    plt.ylim([0, 1.1])
    
plt.figure(figsize=(8, 6))
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
# plt.xlim([-700000, 700000])
plt.show()